In [2]:
import pandas as pd
import numpy as np

# Constituents Data
## Description

In [2]:
constituents = pd.read_excel("../crsp_data/constituents.xlsx")
constituents.head()

,Date,Ticker,Return,RussellMC,RMIDC_WT
0,19781229,4165A,NaN,Y,0.1041
1,19781229,MNST1,NaN,Y,0.1102
2,19781229,ARA,NaN,Y,0.1372
3,19781229,ASA,NaN,Y,0.0887
4,19781229,2982B,NaN,Y,0.1176


In [3]:
grouped = constituents.groupby("Date")
describe = grouped.count()[["Ticker", "RussellMC"]].copy()
describe["WT_SUM"] = constituents.dropna(subset=["Ticker"]).groupby("Date").sum().RMIDC_WT
describe

,Ticker,RussellMC,WT_SUM
Date,,,
19781229,755,800,94.804700
19790330,761,800,95.503700
19790629,762,800,95.484000
19790928,761,800,95.547300
19791231,764,800,96.372800
...,...,...,...
20220831,826,826,1.000013
20220930,825,825,0.999979
20221031,817,817,1.000008


In [4]:
constituents["DateTime"] = pd.to_datetime(constituents["Date"].astype(str))
constituents["Year"] = [d.year for d in constituents["DateTime"]]
constituents["Month"] = [d.month for d in constituents["DateTime"]]

In [5]:
constituents_q = constituents.loc[constituents['Month'].isin([3, 6, 9, 12]), :].copy()
constituents_q.to_excel("../crsp_analysis/constituents_quarterly.xlsx")

In [41]:
members_ls = constituents_q.Ticker.drop_duplicates()
members_ls.to_csv("../crsp_analysis/constituents_list.csv", index=False)

## Returns

In [3]:
returns = pd.read_csv("../crsp_data/returns.csv")
returns

/Users/zhangzixin/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,TICKER,PERMNO,HdrCUSIP,HdrCUSIP9,CUSIP,CUSIP9,MthCalDt,MthPrc,MthCap,MthRet,MthRetx,DisFacPr
0,ACF,10006,00080010,000800102,00080010,000800102,19700130,46.3750,"261,601.38",-0.053571,-0.053571,NaN
1,ACF,10006,00080010,000800102,00080010,000800102,19700227,48.1250,"271,473.13",0.051272,0.037736,0.0
2,ACF,10006,00080010,000800102,00080010,000800102,19700331,49.2500,"277,819.25",0.023377,0.023377,NaN
3,ACF,10006,00080010,000800102,00080010,000800102,19700430,45.5000,"256,665.50",-0.076142,-0.076142,NaN
4,ACF,10006,00080010,000800102,00080010,000800102,19700529,38.5000,"217,178.50",-0.139744,-0.153846,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1259552,TSLA,93436,88160R10,88160R101,88160R10,88160R101,20220831,275.6100,"863,615,666.70",-0.072489,-0.072489,2.0
1259553,TSLA,93436,88160R10,88160R101,88160R10,88160R101,20220930,265.2500,"837,659,500.00",-0.037589,-0.037589,NaN
1259554,TSLA,93436,88160R10,88160R101,88160R10,88160R101,20221031,227.5400,"718,514,890.08",-0.142168,-0.142168,NaN
1259555,TSLA,93436,88160R10,88160R101,88160R10,88160R101,20221130,194.7000,"614,814,314.40",-0.144326,-0.144326,NaN


In [7]:
grouped = returns.groupby("TICKER")
ls = []
for _, group in grouped:
    # group["QthRet"] = (group["MthRet"] + 1).rolling(3).apply(np.prod, raw=True).shift(-2) - 1
    group["QthRet"] = (group["MthRet"] + 1).rolling(3).apply(np.prod, raw=True) - 1
    ls.append(group)

returns = pd.concat(ls)

In [8]:
returns["Datetime"] = pd.to_datetime(returns["MthCalDt"].astype(str))
returns["Year"] = [d.year for d in returns["Datetime"]]
returns["Month"] = [d.month for d in returns["Datetime"]]

In [10]:
returns.columns

Index(['TICKER', 'SecurityNm', 'MthCalDt', 'MthPrcDt', 'MthCap', 'MthPrevCap',
       'MthRet', 'QthRet', 'Datetime', 'Year', 'Month'],
      dtype='object')

In [11]:
returns = returns[['TICKER', 'MthCap', 'QthRet', 'Year', 'Month']].copy()

In [87]:
from datetime import datetime

In [88]:
constituents_q["DateQ"] = [datetime.strftime(d, "%m/%d/%Y") for d in constituents_q["DateTime"]]
constituents_q

,Date,Ticker,Return,RussellMC,RMIDC_WT,DateTime,Year,Month,DateQ
0,19781229,4165A,NaN,Y,0.104100,1978-12-29,1978,12,12/29/1978
1,19781229,MNST1,NaN,Y,0.110200,1978-12-29,1978,12,12/29/1978
2,19781229,ARA,NaN,Y,0.137200,1978-12-29,1978,12,12/29/1978
3,19781229,ASA,NaN,Y,0.088700,1978-12-29,1978,12,12/29/1978
4,19781229,2982B,NaN,Y,0.117600,1978-12-29,1978,12,12/29/1978
...,...,...,...,...,...,...,...,...,...
369689,20221230,ESTC,-0.232468,Y,0.000427,2022-12-30,2022,12,12/30/2022
369690,20221230,LYB,0.156815,Y,0.002285,2022-12-30,2022,12,12/30/2022
369691,20221230,QGEN,-0.637577,Y,0.001211,2022-12-30,2022,12,12/30/2022
369692,20221230,CPA,-0.180029,Y,0.000255,2022-12-30,2022,12,12/30/2022


In [89]:
constituents_q.to_excel("../clean_data/constituents_quarterly.xlsx")

In [12]:
index_return = constituents_q.merge(returns, left_on=["Ticker", "Year", "Month"], right_on=['TICKER', "Year", "Month"], how="left")
index_return

,Date,Ticker,Return,RussellMC,RMIDC_WT,DateTime,Year,Month,TICKER,MthCap,QthRet
0,19781229,4165A,NaN,Y,0.104100,1978-12-29,1978,12,NaN,NaN,NaN
1,19781229,MNST1,NaN,Y,0.110200,1978-12-29,1978,12,NaN,NaN,NaN
2,19781229,ARA,NaN,Y,0.137200,1978-12-29,1978,12,ARA,"355,500.00",-0.153256
3,19781229,ASA,NaN,Y,0.088700,1978-12-29,1978,12,ASA,"229,200.00",-0.138296
4,19781229,2982B,NaN,Y,0.117600,1978-12-29,1978,12,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
146056,20221230,ESTC,-0.232468,Y,0.000427,2022-12-30,2022,12,NaN,NaN,NaN
146057,20221230,LYB,0.156815,Y,0.002285,2022-12-30,2022,12,NaN,NaN,NaN
146058,20221230,QGEN,-0.637577,Y,0.001211,2022-12-30,2022,12,NaN,NaN,NaN
146059,20221230,CPA,-0.180029,Y,0.000255,2022-12-30,2022,12,NaN,NaN,NaN


In [74]:
index_return.to_excel("../crsp_analysis/returns_index.xlsx")

In [83]:
index_return["MthRet_WT"] = index_return.RMIDC_WT * index_return.MthRet

In [84]:
grouped = index_return.groupby("Date")
describe = grouped.count()[["RussellMC", "Ticker", "MthRet"]].copy()
grouped = index_return.dropna(subset=["MthRet"]).groupby("Date")
describe["WT_SUM"] = grouped.sum().RMIDC_WT
describe["MthMet_equal"] = grouped.mean().MthRet
describe["MthMet_weighted"] = grouped.sum().MthRet_WT/100
describe.to_excel("../crsp_analysis/returns_index_return_monthly.xlsx")

## Using Close Price to calculate returns

In [88]:
ClosePrice = pd.read_csv("../crsp_data/QUOTEMEDIA_PRICES.csv")

# Index free cash flow on invested capital

In [24]:
FCFROIC = pd.read_excel("../clean_data/FCFROIC_ADJ_YEARLY_0109.xlsx")
FCFROIC

,Unnamed: 0,Global Company Key,Data Date,Fiscal Year,Fiscal Quarter,Ticker Symbol,FCFROIC,FCFROIC_ADJ,avg_fcfroic_min5,avg_fcfroic_5,avg_fcfroic_7,avg_fcfroic,avg_fcfroic_cal_min5,avg_fcfroic_cal_5,avg_fcfroic_cal_7,avg_fcfroic_cal
0,0,1004,1971-05-31,1970,4,AIR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1004,1972-05-31,1971,4,AIR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1004,1973-05-31,1972,4,AIR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1004,1974-05-31,1973,4,AIR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1004,1975-05-31,1974,4,AIR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61590,61590,328795,2018-12-31,2018,4,ACA,0.045054,0.046591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61591,61591,328795,2019-12-31,2019,4,ACA,0.144107,0.158797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61592,61592,328795,2020-12-31,2020,4,ACA,0.082630,0.088033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61593,61593,328795,2021-12-31,2021,4,ACA,0.030491,0.031091,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
FCFROIC["next year"] = FCFROIC["Fiscal Year"] + 1

In [69]:
NAME = "avg_fcfroic"
# 'FCFROIC', 'FCFROIC_ADJ',
#        'avg_fcfroic_min5', 'avg_fcfroic_5', 'avg_fcfroic_7', 'avg_fcfroic',
#        'avg_fcfroic_cal_min5', 'avg_fcfroic_cal_5', 'avg_fcfroic_cal_7',
#        'avg_fcfroic_cal'
FCFROIC_u = FCFROIC[['Data Date', 'Fiscal Year',
       'Fiscal Quarter', 'Ticker Symbol', NAME, 'next year']].copy()

In [70]:
index_fcfroic = index_return.merge(FCFROIC_u, left_on=["Ticker", "Year"], right_on=["Ticker Symbol", "next year"], how="left")
index_fcfroic

,Date,Ticker,Return,RussellMC,RMIDC_WT,DateTime,Year,Month,TICKER,MthCap,QthRet,Data Date,Fiscal Year,Fiscal Quarter,Ticker Symbol,avg_fcfroic,next year
0,19781229,4165A,NaN,Y,0.104100,1978-12-29,1978,12,NaN,NaN,NaN,1977-12-31,1977.0,4.0,4165A,NaN,1978.0
1,19781229,MNST1,NaN,Y,0.110200,1978-12-29,1978,12,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
2,19781229,ARA,NaN,Y,0.137200,1978-12-29,1978,12,ARA,"355,500.00",-0.153256,NaT,NaN,NaN,NaN,NaN,NaN
3,19781229,ASA,NaN,Y,0.088700,1978-12-29,1978,12,ASA,"229,200.00",-0.138296,1977-11-30,1977.0,4.0,ASA,NaN,1978.0
4,19781229,2982B,NaN,Y,0.117600,1978-12-29,1978,12,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146057,20221230,ESTC,-0.232468,Y,0.000427,2022-12-30,2022,12,NaN,NaN,NaN,2022-04-30,2021.0,4.0,ESTC,NaN,2022.0
146058,20221230,LYB,0.156815,Y,0.002285,2022-12-30,2022,12,NaN,NaN,NaN,2021-12-31,2021.0,4.0,LYB,0.203045,2022.0
146059,20221230,QGEN,-0.637577,Y,0.001211,2022-12-30,2022,12,NaN,NaN,NaN,2021-12-31,2021.0,4.0,QGEN,0.061409,2022.0
146060,20221230,CPA,-0.180029,Y,0.000255,2022-12-30,2022,12,NaN,NaN,NaN,2021-12-31,2021.0,4.0,CPA,0.108201,2022.0


In [18]:
# his_mean = index_fcfroic.groupby("DateTime").mean()
# data = his_mean["avg_fcfroic"].dropna()
# data.to_excel("../clean_data/historical_mean_fcfroic.xlsx")

# Return on Capital

In [19]:
ecoc = pd.read_excel("../clean_data/ECoC_0109_dropna.xlsx")
ecoc

,Unnamed: 0,beta,name,DATES,GT10 Govt,rf,coe
0,60,1.243701,A,1984-03-30,12.47095,0.12471,0.199332
1,60,1.737492,AAC,1984-03-30,12.47095,0.12471,0.228959
2,60,1.192654,AAL,1984-03-30,12.47095,0.12471,0.196269
3,60,1.469890,AAR,1984-03-30,12.47095,0.12471,0.212903
4,60,0.999707,ABAN,1984-03-30,12.47095,0.12471,0.184692
...,...,...,...,...,...,...,...
756009,247,1.276358,ZBRA,1999-10-29,6.02400,0.06024,0.136822
756010,247,1.062015,ZION,1999-10-29,6.02400,0.06024,0.123961
756011,247,1.043467,ZLC,1999-10-29,6.02400,0.06024,0.122848
756012,247,0.433167,ZNT,1999-10-29,6.02400,0.06024,0.086230


In [20]:
index_fcfroic

,Date,Ticker,Return,RussellMC,RMIDC_WT,DateTime,Year,Month,TICKER,MthCap,...,FCFROIC_ADJ,avg_fcfroic_min5,avg_fcfroic_5,avg_fcfroic_7,avg_fcfroic,avg_fcfroic_cal_min5,avg_fcfroic_cal_5,avg_fcfroic_cal_7,avg_fcfroic_cal,next year
0,19781229,4165A,NaN,Y,0.104100,1978-12-29,1978,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1978.0
1,19781229,MNST1,NaN,Y,0.110200,1978-12-29,1978,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19781229,ARA,NaN,Y,0.137200,1978-12-29,1978,12,ARA,"355,500.00",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,19781229,ASA,NaN,Y,0.088700,1978-12-29,1978,12,ASA,"229,200.00",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1978.0
4,19781229,2982B,NaN,Y,0.117600,1978-12-29,1978,12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146057,20221230,ESTC,-0.232468,Y,0.000427,2022-12-30,2022,12,NaN,NaN,...,0.021235,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022.0
146058,20221230,LYB,0.156815,Y,0.002285,2022-12-30,2022,12,NaN,NaN,...,0.234606,0.203045,0.159609,0.189975,0.203045,0.233534,0.170503,0.209203,0.233534,2022.0
146059,20221230,QGEN,-0.637577,Y,0.001211,2022-12-30,2022,12,NaN,NaN,...,0.111599,0.061409,0.067087,0.066932,0.061409,0.070258,0.077308,0.077002,0.070258,2022.0
146060,20221230,CPA,-0.180029,Y,0.000255,2022-12-30,2022,12,NaN,NaN,...,0.013406,0.108201,0.076716,0.092422,0.108201,0.140770,0.095076,0.114266,0.140770,2022.0


In [28]:
ecoc["Datetime"] = pd.to_datetime(ecoc["DATES"])
ecoc["Year"] = [d.year for d in ecoc["Datetime"]]
ecoc["Month"] = [d.month for d in ecoc["Datetime"]]
ecoc

,Unnamed: 0,beta,name,DATES,GT10 Govt,rf,coe,Datetime,Year,Month
0,60,1.243701,A,1984-03-30,12.47095,0.12471,0.199332,1984-03-30,1984,3
1,60,1.737492,AAC,1984-03-30,12.47095,0.12471,0.228959,1984-03-30,1984,3
2,60,1.192654,AAL,1984-03-30,12.47095,0.12471,0.196269,1984-03-30,1984,3
3,60,1.469890,AAR,1984-03-30,12.47095,0.12471,0.212903,1984-03-30,1984,3
4,60,0.999707,ABAN,1984-03-30,12.47095,0.12471,0.184692,1984-03-30,1984,3
...,...,...,...,...,...,...,...,...,...,...
756009,247,1.276358,ZBRA,1999-10-29,6.02400,0.06024,0.136822,1999-10-29,1999,10
756010,247,1.062015,ZION,1999-10-29,6.02400,0.06024,0.123961,1999-10-29,1999,10
756011,247,1.043467,ZLC,1999-10-29,6.02400,0.06024,0.122848,1999-10-29,1999,10
756012,247,0.433167,ZNT,1999-10-29,6.02400,0.06024,0.086230,1999-10-29,1999,10


In [61]:
ave_ecoc.columns

Index(['Unnamed: 0', 'beta', 'name', 'DATES', 'GT10 Govt', 'rf', 'coe',
       'Datetime', 'Year', 'Month', '10y_avg'],
      dtype='object')

In [62]:
avg_ecoc = ave_ecoc[['name', 'Year', 'Month', '10y_avg']].copy()

In [72]:
data_all = index_fcfroic.merge(avg_ecoc, left_on=["Ticker", "Year", "Month"], right_on=["name", "Year", "Month"], how="left")
data_all

,Date,Ticker,Return,RussellMC,RMIDC_WT,DateTime,Year,Month,TICKER,MthCap,QthRet,Data Date,Fiscal Year,Fiscal Quarter,Ticker Symbol,avg_fcfroic,next year,name,10y_avg
0,19781229,4165A,NaN,Y,0.104100,1978-12-29,1978,12,NaN,NaN,NaN,1977-12-31,1977.0,4.0,4165A,NaN,1978.0,NaN,NaN
1,19781229,MNST1,NaN,Y,0.110200,1978-12-29,1978,12,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19781229,ARA,NaN,Y,0.137200,1978-12-29,1978,12,ARA,"355,500.00",-0.153256,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,19781229,ASA,NaN,Y,0.088700,1978-12-29,1978,12,ASA,"229,200.00",-0.138296,1977-11-30,1977.0,4.0,ASA,NaN,1978.0,NaN,NaN
4,19781229,2982B,NaN,Y,0.117600,1978-12-29,1978,12,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146057,20221230,ESTC,-0.232468,Y,0.000427,2022-12-30,2022,12,NaN,NaN,NaN,2022-04-30,2021.0,4.0,ESTC,NaN,2022.0,NaN,NaN
146058,20221230,LYB,0.156815,Y,0.002285,2022-12-30,2022,12,NaN,NaN,NaN,2021-12-31,2021.0,4.0,LYB,0.203045,2022.0,NaN,NaN
146059,20221230,QGEN,-0.637577,Y,0.001211,2022-12-30,2022,12,NaN,NaN,NaN,2021-12-31,2021.0,4.0,QGEN,0.061409,2022.0,NaN,NaN
146060,20221230,CPA,-0.180029,Y,0.000255,2022-12-30,2022,12,NaN,NaN,NaN,2021-12-31,2021.0,4.0,CPA,0.108201,2022.0,NaN,NaN


In [73]:
quality = data_all.copy()

quality["ind"] = np.nan
quality.loc[(quality[NAME] >=
            quality["10y_avg"]), "ind"] = 1
quality.loc[(quality[NAME] <
            quality["10y_avg"]), "ind"] = 0

In [74]:
quality.to_excel("../crsp_analysis/quality_0210_{NAME}.xlsx".format(NAME=NAME))

In [43]:
quality.columns

Index(['Date', 'Ticker', 'Return', 'RussellMC', 'RMIDC_WT', 'DateTime', 'Year',
       'Month', 'TICKER', 'MthCap', 'QthRet', 'Data Date', 'Fiscal Year',
       'Fiscal Quarter', 'Ticker Symbol', 'avg_fcfroic_min5', 'next year',
       'name', '10y_avg', 'ind'],
      dtype='object')

In [75]:
high_q = quality[quality['ind'] == 1].copy()
high_ret = high_q.groupby("Date").mean().QthRet
low_q = quality[quality['ind'] == 0].copy()
low_ret = low_q.groupby("Date").mean().QthRet

In [76]:
total_ret = quality.groupby("Date").mean().QthRet

In [77]:
ret_all = pd.DataFrame()
ret_all["high quality"] = high_ret
ret_all["low quality"] = low_ret
ret_all["total"] = total_ret


In [78]:
cumulative = (ret_all + 1).cumprod() - 1
ret_all[["high cum", "low cum", "total cum"]] = cumulative

In [79]:
ret_all.to_excel('../crsp_analysis/ret_high_low_all.xlsx')

In [80]:
grouped = quality.groupby("Date")
grouped.mean().dropna(how="all").to_excel("../crsp_analysis/quality_rate.xlsx")

In [81]:
grouped.count().to_excel("../crsp_analysis/quality_num.xlsx")

In [82]:
high_quality = quality[quality['ind'] == 1].copy()
grouped = high_quality.groupby("Date")
grouped.count().to_excel("../crsp_analysis/quality_hq_num.xlsx")